In [1]:
%pip install gurobipy -qU

Note: you may need to restart the kernel to use updated packages.


In [2]:
import gurobipy as gp
from gurobipy import GRB

In [3]:
periods = range(3)
demands = [15,35,60]
capacity = 40
stockcost = 3
suppliercost = 5

In [4]:
# Model
m = gp.Model("stock")

#Variables
produce = m.addVars(periods, name = "produce" )
buy = m.addVars(periods, name = "buy")
stock = m.addVars(periods, name = "stock")

Restricted license - for non-production use only - expires 2025-11-24


In [5]:
# The objective is to minimize the costs
m.setObjective(sum(buy[p]*suppliercost for p in periods) + sum(stock[p]*stockcost for p in periods), GRB.MINIMIZE)

In [6]:
# Constraints on the production per time period
for p in periods:
  m.addConstr(produce[p] <= 40)

In [7]:
# Constraints on the production per time period
m.addConstr(stock[0] - produce[0] + demands[0] == 0)
for p in range(1,3):
    m.addConstr(stock[p] -stock [p-1] - produce[p] + demands[p] == 0)

In [8]:
m.write("stock.lp")
f = open("stock.lp","r")
contents = f.read()
print(contents)

\ Model stock
\ LP format - for model browsing. Use MPS format to capture full model detail.
Minimize
  5 buy[0] + 5 buy[1] + 5 buy[2] + 3 stock[0] + 3 stock[1] + 3 stock[2]
Subject To
 R0: produce[0] <= 40
 R1: produce[1] <= 40
 R2: produce[2] <= 40
 R3: - produce[0] + stock[0] = -15
 R4: - produce[1] - stock[0] + stock[1] = -35
 R5: - produce[2] - stock[1] + stock[2] = -60
Bounds
End



In [9]:
# Solve
m.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 20.04.6 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 6 rows, 9 columns and 11 nonzeros
Model fingerprint: 0x0c34400b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 5e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+01, 6e+01]
Presolve removed 6 rows and 9 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0500000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.050000000e+02


In [10]:
def printSolution():
    if m.status == GRB.OPTIMAL:
        print('\nCost: %g' % m.ObjVal)
        for p in periods:
            print("Buy = ", buy[p].X, " Produce = ", produce[p].X, " Stock = ", stock[p].X)
    else:
        print('No solution')

In [11]:
printSolution()


Cost: 105
Buy =  0.0  Produce =  30.0  Stock =  15.0
Buy =  0.0  Produce =  40.0  Stock =  20.0
Buy =  0.0  Produce =  40.0  Stock =  0.0
